In [1]:
#check the data docs in play and down folders
dbutils.fs.ls("/FileStore/tables/play")

In [2]:
#assign schema for play table
from pyspark.sql.types import *
schema_play = StructType([
  StructField("uid", StringType(), True),
  StructField("device", StringType(), True),
  StructField("song_id", StringType(), True),
  StructField("song_type", StringType(), True),
  StructField("song_name", StringType(), True),
  StructField("singer", StringType(), True),
  StructField("play_time", StringType(), True),
  StructField("song_length", StringType(), True),
  StructField("paid_flag", StringType(), True),
  StructField("date", StringType(), False)
])

schema_df = StructType([
  StructField("uid", StringType(), True),
  StructField("device", StringType(), True),
  StructField("song_id", StringType(), True),
  StructField("song_type", StringType(), True),
  StructField("song_name", StringType(), True),
  StructField("singer", StringType(), True),
  StructField("play_time", StringType(), True),
  StructField("song_length", StringType(), True),
  StructField("paid_flag", StringType(), True)
])

In [3]:
#load in data and compile into play table
from pyspark.sql.functions import lit

play = spark.createDataFrame([],schema=schema_play)
for path, name, _ in dbutils.fs.ls("/FileStore/tables/play"):
  date = name.split("_")[0]
  df = spark.read.csv(path=path, sep="\t", schema=schema_df)
  df = df.withColumn("date", lit(date))
  play = play.unionAll(df)
play.createOrReplaceTempView("play")

#two source files are corrupt
#20170424_1_play_log_tar-5ebb6.gz
#20170424_2_play_log_tar-0b542.gz

In [4]:
#assign schema for down table
from pyspark.sql.types import *
schema_down = StructType([
  StructField("uid", StringType(), True),
  StructField("device", StringType(), True),
  StructField("song_id", StringType(), True),
  StructField("song_name", StringType(), True),
  StructField("singer", StringType(), True),
  StructField("paid_flag", StringType(), True),
  StructField("date", StringType(), False)
])

schema_df2 = StructType([
  StructField("uid", StringType(), True),
  StructField("device", StringType(), True),
  StructField("song_id", StringType(), True),
  StructField("song_name", StringType(), True),
  StructField("singer", StringType(), True),
  StructField("paid_flag", StringType(), True)
])

In [5]:
#load in data and compile into down table
from pyspark.sql.functions import lit
down = spark.createDataFrame([],schema=schema_down)
for path, name, _ in dbutils.fs.ls("/FileStore/tables/down"):
  date = name.split("_")[0]
  df = spark.read.csv(path=path, sep="\t", schema=schema_df2)
  df = df.withColumn("date", lit(date))
  down = down.unionAll(df)
down.createOrReplaceTempView("down")

In [6]:
%sql --count # of rows in play table
select count(*) from play

In [7]:
%sql --count # of rows in down table
select count(*) from down

In [8]:
%sql
select * from play limit 20

In [9]:
#cast the data types
play_casted=sqlContext.sql("""
select trim(uid) as uid, trim(song_id) as song_id, trim(song_name) as song_name, trim(singer) as singer, 
       cast(trim(play_time) as int) as play_time, cast(trim(song_length) as int) as song_length, trim(paid_flag) as paid_flag,
       TO_DATE(CAST(UNIX_TIMESTAMP(date, 'yyyyMMdd') AS TIMESTAMP)) as date
from play
where trim(uid) is not null and trim(song_id) is not null and cast(trim(play_time) as int) is not null
""")
play_casted.createOrReplaceTempView("play_casted")

In [10]:
%sql
select * from play_casted where (song_length=0 or song_length=1 or song_length is null) and paid_flag is null

In [11]:
%sql
--count # of rows for each length of uid
select length(uid), count(*)
from play_casted
group by length(uid)
order by count(*) desc

--uid's with length of 6-9 seem normal

In [12]:
%sql
--check length(uid)<6
select uid, count(*)
from play_casted
where length(uid)<6
group by uid
order by count(*)

--uid=0 has 1170088 rows of records
--need to identify if it's robot
--length(uid)=5 can also be considered normal

In [13]:
%sql --remove it
select uid
from play_casted
where length(uid)=153

In [14]:
%sql
--substring the misformatted uid
select uid,substring(uid,-9) as new_uid
from play_casted
where length(uid)>500

In [15]:
def trimmer(string):
  uid=""
  for c in string:
    if c in ["1","2","3","4","5","6","7","8","9","0"]: 
      uid+=c
  return uid
sqlContext.udf.register("trimmer", trimmer)

In [16]:
%sql
select uid, substring(uid,-9), length(substring(uid,-9)), trimmer(substring(uid,-9)),  length(trimmer(substring(uid,-9))) 
from play_casted
where length(uid)>500

In [17]:
%sql
--check the length of song_id
select length(song_id), count(*)
from play_casted
group by length(song_id)
order by count(*) desc

In [18]:
%sql
--check the misformatted song_id
select song_id, song_name, song_length
from play_casted
where length(song_id)>100

In [19]:
#create table for cleaned play
play_cleaned=sqlContext.sql("""
select (case when length(uid)>500 then trimmer(substring(uid,-9)) else uid end) as uid, 
       song_id, song_name, singer,
       play_time, song_length,
       date
from play_casted
where length(song_id)<30 and 
      (length(uid)<10 or length(uid)>500) and 
      not ((song_length=0 or song_length=1 or song_length is null) and paid_flag is null)
""")
play_cleaned.createOrReplaceTempView("play_cleaned")

In [20]:
%sql
select * from down limit 20

In [21]:
%sql
--count # of rows for each length of uid
select length(trim(uid)),count(*)
from down
group by length(trim(uid))
order by count(*) desc

--length of 6-9 is normal
--length=1 is uid=0
--length>500, misformatting, need to do substring

In [22]:
%sql
--check the length of song_id
select length(trim(song_id)),count(*)
from down
group by length(trim(song_id))
order by count(*) desc


In [23]:
%sql
--check rows with null uid or song_id
select *
from down
where uid is null or song_id is null

--null records

In [24]:
#create table for cleaned down
down_cleaned=sqlContext.sql("""
select (case when length(uid)>500 then trimmer(substring(uid,-9)) else uid end) as uid, 
       song_id, song_name, singer,
       TO_DATE(CAST(UNIX_TIMESTAMP(date, 'yyyyMMdd') AS TIMESTAMP)) as date
from (
     select trim(uid) as uid, trim(song_id) as song_id, trim(song_name) as song_name, trim(singer) as singer, date
     from down
     where uid is not null and song_id is not null
     )
""")
down_cleaned.createOrReplaceTempView("down_cleaned")

In [25]:
%sql
--show song_id="", see multiple song_name's
select *
from play_cleaned
where length(song_id)=0



In [26]:
%sql
--check song_id's and song_length's for "纸月亮" 
select song_id, song_name, singer, song_length, count(*)
from play_cleaned
where song_name like "纸月亮"
group by song_id, song_name, singer, song_length
order by song_id, song_length

--there are 5 distinct song_id for 1 song
--I am going to use the mode(song_length) as the song_length

In [27]:
%sql
--play has 1,679,872 song_id's and 1,760,049 song_name's and 2,255,855 name&singer 
select count(distinct song_id) as num_id, count(distinct song_name) as num_name, count(distinct song_name,singer) as num_name_singer
from play_cleaned

In [28]:
%sql
--count # of song_ids for each song_name&singer
select song_name, singer, count(distinct song_id) as num_ids
from play_cleaned
group by song_name, singer
order by num_ids desc

In [29]:
%sql
--265,937(11.8%) song_name&singer with >1 song_id 
select count(*)
from (
select song_name, singer, count(distinct song_id) as num_ids
from play_cleaned
group by song_name, singer
  )
where num_ids>1


In [30]:
%sql
--122,083(5.4%) name&singer with >1 song_id after removing song_id="0","-1", "18446744073709551615", ""
select count(*)
from (
select song_name, singer, count(distinct song_id) as num_ids
from play_cleaned
where song_id not in ("0","-1", "18446744073709551615", "")
group by song_name, singer
  )
where num_ids>1


In [31]:
%sql
--32,754(1.95%) song_id's with >1 song_names
--select count(*) from (
select song_id, count(distinct song_name) as num_names
from play_cleaned
group by song_id
order by num_names desc
--  )
--where num_names>1


In [32]:
%sql
--check the song contents with song_id of "0", "-1", "18446744073709551615", "",  
select *
from play_casted
where song_id="1563"

In [33]:
%sql
--check the # of song_id's of "负心的人"
select song_name, count(distinct song_id)
from play_casted
where song_name="负心的人"
group by song_name

In [34]:
%sql
--74,545,005(45.9%) rows for song_id's with >1 name&singer
select count(*)
from play_cleaned
where song_id in (
  select song_id 
  from play_cleaned
  group by song_id
  having count(distinct song_name,singer)>1
  )

In [35]:
%sql
--87,568,293(53.9%) rows for song_id's with >1 name&singer
select count(*)
from play_cleaned
where (song_name, singer) in (
  select song_name, singer 
  from play_cleaned
  where song_id not in ("0", "-1", "18446744073709551615", "")
  group by song_name, singer
  having count(distinct song_id)>1
  )

In [36]:
%sql
--down has 739,682 song_id's and 484,907 song_name's and 601,918 name&singer
select count(distinct song_id) as num_id, count(distinct song_name) as num_name, count(distinct song_name,singer) as num_name_singer
from down_cleaned

In [37]:
%sql
--count # of song_names for each song_id for down table
--36,986 song_id's with >1 song_names, 5% of total id's
--select count(*) from (
select song_id, count(distinct song_name) as num_names
from down_cleaned
group by song_id
order by num_names desc
--  )
--where num_names>1

In [38]:
play_u=sqlContext.sql("""
select song_id, song_name, singer
from play_cleaned
""")

down_u=sqlContext.sql("""
select song_id, song_name, singer
from down_cleaned
""")

df=play_u.union(down_u)
df.createOrReplaceTempView("df")

In [39]:
%sql
--there are 162,543,495 rows in play, 7,737,067 rows in down, and 170,280,562 rows in df
select count(*)
from play_cleaned
--from down_cleaned
--from df

In [40]:
%sql
--1,041,125 rows name and singer is null, 0.6%
--1,042,833 rows name is null
--1,126,398 rows name or singer is null, 0.66%
select count(*)
from df
--where song_name is null
where song_name is null and singer is null
--where song_name is null or singer is null

--I decide to remove the rows with null song_name or singer

In [41]:
%sql
--92,469 rows name and singer is ""
--93,540 rows name is ""
--109,438 rows name or singer is "", 0.06%
select count(*)
from df
--where song_name like ""
--where song_name like "" and singer like ""
where song_name like "" or singer like ""

--I decide to remove the rows with "" song_name or singer

In [42]:
sid=sqlContext.sql("""
select song_name, singer, row_number () over (order by song_name, singer) as sid
from df
where song_name is not null and singer is not null and song_name not like "" and singer not like ""
group by song_name, singer
""")
sid.createOrReplaceTempView("sid")
sid.write.parquet("/FileStore/sid.parquet")

In [43]:
%sql
select * from sid limit 100

In [44]:
%sql --2,309,630 sid's
select count(*) from sid

In [45]:
play_sid=sqlContext.sql("""
select cast(uid as int) as uid, cast(sid as int) as sid, play_time, song_length, date
from play_cleaned p join
     sid s
     on p.song_name=s.song_name and p.singer=s.singer
""")
play_sid.createOrReplaceTempView("play_sid")

In [46]:
down_sid=sqlContext.sql("""
select cast(uid as int) as uid, cast(sid as int) as sid, date
from down_cleaned d join
     sid s
     on d.song_name=s.song_name and d.singer=s.singer
""")
down_sid.createOrReplaceTempView("down_sid")

In [47]:
%sql --6,592,790 rows in down_sid
select count(*) as num_rows, count(distinct uid) as num_users, count(distinct sid) as num_songs
from down_sid

In [48]:
%sql
select sid, song_length
from (
  select sid, song_length, rank() over (partition by sid order by count(*)desc) as rank
  from play_sid
  where song_length>0
  group by sid, song_length
  )r
where r.rank=1
order by sid


In [49]:
song_length=sqlContext.sql("""
select sid, song_length
from (
  select sid, song_length, rank() over (partition by sid order by count(*)desc) as rank
  from play_sid
  where song_length>0
  group by sid, song_length
  )r
where r.rank=1
order by sid
""")
song_length.createOrReplaceTempView("song_length")
song_length.write.parquet("/FileStore/song_length.parquet")

In [50]:
#2,146,170 songs
song_length.count()

In [51]:
play_sid_sl=sqlContext.sql("""
select uid, p.sid, play_time, s.song_length, date
from play_sid p join
     song_length s
     on p.sid=s.sid
""")
play_sid_sl.createOrReplaceTempView("play_sid_sl")
sqlContext.cacheTable("play_sid_sl")

In [52]:
%sql --161,941,162 rows in play
select count(*) as num_rows, count(distinct uid) as num_users, count(distinct sid) as num_songs
from play_sid_sl